## Esercizi OCaml
### Esercizi in OCaml presi da vari esami del corso Paradigmi di programmazione dell'Università di Pisa.

**Esercizio 1**

Si consideri il seguente tipo di dato che descrive alberi n-ari con nodi associati a valori interi, dove la
ntree list si assume essere non vuota:


type ntree = Node of int * ntree list
| Leaf of int;;


Definire, usando i costrutti di programmazione funzionale in OCaml, una funzione compact con tipo
compact : ntree -> ntree
tale che compact t restituisca un albero t’ ottenuto sostituendo ogni nodo Node (n,[Leaf n1;...;Leaf nk])
(ossia che non ha figli di tipo Node) contenuto in t, con un nodo Node (n,[Leaf (n1+...+nk)]). 

Esempi:


compact (Node(10,[Leaf 2; Leaf 3; Leaf 4])) =
Node (10,[Leaf 9])


compact (Node(10,[Leaf 2; Node (3, [Leaf 5; Leaf 3]); Leaf 4])) =
Node (10,[Leaf 2; Node (3, [Leaf 8]); Leaf 4])


compact (Node(10,[])) =
Node (10,[Leaf 0])

In [1]:
type ntree = 
    | Leaf of int
    | Node of int * ntree list ;;

type ntree = Leaf of int | Node of int * ntree list


In [2]:
let rec compact nt =
  match nt with
  | Leaf v -> Leaf v (* caso base *)
  | Node (v, children) -> 
      if children = [] then Node (v, [Leaf 0]) (* lista vuota *)
      else 
        let is_leaf = function Leaf _ -> true | _ -> false in (* se Leaf true, false altrimenti *)
        if List.for_all is_leaf children then (* controlla che nella lista ci siano solo tipi Leaf *)
              let sum = List.fold_left (fun acc child -> 
                      match child with
                      | Leaf l -> acc + l
                      | _ -> acc) 0 children
          in Node (v, [Leaf sum])
        else
          Node (v, List.map compact children) ;;
          
compact (Node(10,[Leaf 2; Leaf 3; Leaf 4]));;

compact (Node(10,[Leaf 2; Node (3, [Leaf 5; Leaf 3]); Leaf 4])) ;;

compact (Node(10,[])) 

val compact : ntree -> ntree = <fun>


- : ntree = Node (10, [Leaf 9])


- : ntree = Node (10, [Leaf 2; Node (3, [Leaf 8]); Leaf 4])


- : ntree = Node (10, [Leaf 0])


#### Esercizio 2
Assumendo il seguente tipo di dato che descrive alberi binari di interi:


type btree =
| Void
| Node of int * btree * btree


si definisca, usando i costrutti di programmazione funzionale di OCaml, una funzione count con tipo
count : btree -> (int -> bool) -> int
tale che (count bt p) restituisca il numero dei nodi in bt i cui figli soddisfano il predicato p. Ad esempio,
dato il seguente predicato:


let positivo x =
match x with
| Void -> false
| Node (i,_,_) -> i>0


e dato il seguente albero binario:
let bt =
Node (3,
Node (5,Void,Void),
Node (-4,
Node(6,Void,Void),
Node(8,Void,Void)
)
)


abbiamo che count bt positivo = 1 in quanto solo il nodo contenente -4 ha entrambi i figli che soddisfano
il predicato.

In [3]:
type btree = 
    | Void
    | Node of int * btree * btree ;;

type btree = Void | Node of int * btree * btree


In [4]:
let positivo x = 
    match x with
    | Void -> false
    | Node (i, _, _) -> i > 0 ;;

val positivo : btree -> bool = <fun>


In [5]:
let rec count_pos node acc p =
        match node with
        | Void -> acc
        | Node(v, left, right) -> let c = if (p left && p right) then acc+1 else acc 
                                                                 in let left_acc = count_pos left c p
                                                                 in count_pos right left_acc p ;;

let rec count bt p = 
    count_pos bt 0 p ;;
    
let bt = Node (3, Node (5,Void,Void), Node (-4, Node(6,Void,Void), Node(8,Void,Void))) ;;

count bt positivo ;;

val count_pos : btree -> int -> (btree -> bool) -> int = <fun>


val count : btree -> (btree -> bool) -> int = <fun>


val bt : btree =
  Node (3, Node (5, Void, Void),
   Node (-4, Node (6, Void, Void), Node (8, Void, Void)))


- : int = 1


#### Esercizio 3

Senza utilizzare esplicitamente la ricorsione, ma utilizzando funzioni higher-order su liste (dal modulo List),
si definisca in OCaml una funzione split con tipo


split: ‘a list -> (‘a -> bool) -> (‘a list * ‘a list)


in modo che split lis p restituisca una copia (lis1,lis2) tale che la lista lis1 contenga tutti e soli gli
elementi di lis che soddisfano il predicato p e la lista lis2 contenga tutti e soli gli elementi di lis che non
soddisfano il predicato p. 

La soluzione dovrebbe fare in modo che la lista lis venga scandita una sola volta.

In [6]:
let split lis p = 
    (* la funzione aggiunge un elemento a lis1 se il predicato è soddisfatto e a lis2 altrimenti *)
    List.fold_right (fun x (lis1, lis2) -> if (p x) then (x::lis1, lis2) else (lis1, x::lis2)) lis ([], []) ;;
    
    
let l = [1;-2;-3;4;5;6] ;;
split l (function p -> p > 0) ;;
        


val split : 'a list -> ('a -> bool) -> 'a list * 'a list = <fun>


val l : int list = [1; -2; -3; 4; 5; 6]


- : int list * int list = ([1; 4; 5; 6], [-2; -3])


#### Esercizio 4

Definire, usando i costrutti di programmazione funzionale in OCaml, una funzione g con tipo
g : int list -> int * int
che, data una lista non vuota di interi, restituisce la coppia formata dal massimo elemento della lista e
dal numero di volte che esso occorre nella lista stessa. 

Ad esempio: g [1;-4;5;-1;5;-6;5] = (5,3).


L’applicazione di g alla lista vuota causa invece un’eccezione.

In [7]:
let rec max_element lis = 
    match lis with
    | [] -> failwith "Empty list"
    | [x] -> x
    | x::y::lis' -> let max = max_element (y::lis') in if (max < x) then x else max ;;
    
let rec count_elements lis elem = 
    match lis with
    | [] -> 0
    | x::lis' -> if (x = elem) then (count_elements lis' x + 1) else count_elements lis' elem;;
    
let g list =
    match list with
    | [] -> failwith "Empty list"
    | list' -> let max = max_element list' in (max, count_elements list' max) ;;
    
let l = [1;-4;5;-1;5;-6;5] ;;

g l;;

val max_element : 'a list -> 'a = <fun>


val count_elements : 'a list -> 'a -> int = <fun>


val g : 'a list -> 'a * int = <fun>


val l : int list = [1; -4; 5; -1; 5; -6; 5]


- : int * int = (5, 3)


**Esercizio 5**

Assumendo il seguente tipo di dato che descrive interi “colorati” di nero o di rosso:

type colored_int =
| Black of int
| Red of int

si definiscano, usando i costrutti di programmazione funzionale di OCaml, le funzioni swap_colors e seq_len
con tipi
swap_colors : colored_int list -> colored_int list
seq_len : colored_int list -> int
tali che:

    • swap_colors lis restituisca una lista analoga a lis ma in cui gli interi neri sono rossi e viceversa;
    • seq_len lis restituisca la lunghezza della più lunga sequenza di interi dello stesso colore consecutivi
in lis


Ad esempio, data la seguente lista di interi colorati

let lis = [Black 10; Red 5; Red 2; Black 4; Black 1; Black 7; Red 2]
abbiamo che
swap_colors lis;; (* restituisce [Red 10; Black 5; Black 2; Red 4; Red 1; Red 7; Black 2] *)


seq_len lis;;
(* restituisce 3
-- in quanto ci sono tre interi neri in sequenza *)

In [8]:
type colored_int = 
    | Black of int
    | Red of int ;;

type colored_int = Black of int | Red of int


In [9]:
let rec swap_colors lis =
    match lis with 
    | [] -> []
    | Black x::lis' -> Red x::swap_colors lis' 
    | Red x::lis' -> Black x::swap_colors lis' ;;
 
let seq_len lis =
  let rec aux max_len curr_len prev_color = function
        | [] -> max_len
        | Black x ::lis' when prev_color = Some `Black -> aux max_len (curr_len + 1) (Some `Black) lis'
        | Red x ::lis' when prev_color = Some `Red -> aux max_len (curr_len + 1) (Some `Red) lis'
        | Black x ::lis' -> aux (max max_len curr_len) 1 (Some `Black) lis'
        | Red x ::lis' -> aux (max max_len curr_len) 1 (Some `Red) lis'
      in
      aux 0 0 None lis
  

let list = [Black 10; Red 5; Red 2; Black 4; Black 1; Black 7; Red 2] ;;
seq_len list

val swap_colors : colored_int list -> colored_int list = <fun>


val seq_len : colored_int list -> int = <fun>


val list : colored_int list =
  [Black 10; Red 5; Red 2; Black 4; Black 1; Black 7; Red 2]


- : int = 3


**Esercizio 6**

Assumendo il seguente tipo di dato che descrive valute in Euro o Dollari:
type valuta =
| Euro of float
| Dollari of float;;


Definire in OCaml le funzioni inEuro, somma_valute e separa_valute con i seguenti tipi


inEuro : float -> valuta -> valuta
somma_valute : float -> valuta list -> valuta
separa_valute : valuta list -> valuta list * valuta list


tali che:


- inEuro prende un numero reale t che rappresenta il tasso di cambio Euro/Dollaro (attualmente 0,93
Euro per ogni Dollaro), prende x di tipo valuta e restituisce y di tipo valuta. Se x è una valuta
espressa in Dollari, la converte in Euro moltiplicandola per t. Se invece è già in Euro, la restituisce
cosı̀ com’è.


- somma_valute prende un tasso di cambio t (come sopra) e una lista lis di elementi di tipo valuta.
La funzione somma tutti gli elementi di lis restituendo un valore totale espresso in Euro. Eventuali
valori in lis espressi in Dollari devono essere convertiti in Euro usando il tasso t.


- separa_valute prende una lista lis di elementi di tipo valuta e restituisce una coppia di liste
(lis1,lis2) dove lis1 contiene gli elementi di lis espressi in Euro, e lis2 quelli espressi in Dollari.


Ad esempio,
inEuro 0.93 (Euro 2.0) (* calcola Euro 2.0 *);;


inEuro 0.93 (Dollari 2.0) (* calcola Euro 1.86 *);;




considerando la seguente lista di valute:
let lis = [Euro 1.5; Dollari (-1.0); Dollari 2.6; Euro (-2.0);];;


le funzioni danno i seguenti risultati:


- somma_valute 0.93 lis;; (* calcola Euro 0.988 *)

- somma_valute 0.5 lis;;(* calcola Euro 0.3 --- usa tasso Euro/Dollaro = 0.5 *)

- separa_valute lis;; (* calcola la coppia di liste:  (*[Euro 1.5; Euro (-2.0)], [Dollari (-1.0); Dollari 2.6]) *)

In [10]:
type valuta = 
    | Euro of float
    | Dollari of float ;;

type valuta = Euro of float | Dollari of float


In [11]:
let inEuro t x = 
    match x with 
    | Euro m -> Euro m
    | Dollari m -> Euro (m *. t) ;;

val inEuro : float -> valuta -> valuta = <fun>


In [12]:
let rec somma_valute t lis =
  let f sum v = 
    match v with
    | Euro m -> sum +. m
    | Dollari m -> sum +. (m *. t)
    in Euro (List.fold_left f 0. lis) ;;

val somma_valute : float -> valuta list -> valuta = <fun>


In [13]:
let rec separa_valute lis = 
    match lis with 
    | [] -> ([], [])
    | lis' -> ([List.filter (function | Euro _ -> true | _ -> false) lis'], [List.filter (function | Euro _ -> false | _ -> true) lis'])
    
let lis = [Euro 1.5; Dollari (-1.0); Dollari 2.6; Euro (-2.0);];;
separa_valute lis 

val separa_valute : valuta list -> valuta list list * valuta list list =
  <fun>


val lis : valuta list = [Euro 1.5; Dollari (-1.); Dollari 2.6; Euro (-2.)]


- : valuta list list * valuta list list =
([[Euro 1.5; Euro (-2.)]], [[Dollari (-1.); Dollari 2.6]])


**Esercizio 7**

Assumendo il seguente tipo di dato che descrive alberi binari di interi:


type btree =
| Node of int * btree * btree
| Leaf of int


si definiscano, usando i costrutti di programmazione funzionale di OCaml, tre funzioni chiamate profondita,
max_profondita e conta_max con tipi

profondita : btree -> int

max_profondita : btree list -> int

conta_max : btree_list -> int*int

tali che:
   - profondita bt restituisca la profondità dell’albero bt. (NOTA: un albero costituito solo da una foglia
ha profondità pari a 0)
   - max_profondita btlis restituisca il massimo tra le profondità degli alberi contenuti nella lista btlis
(NOTA: se la lista è vuota, restituisce 0)
   - conta_max btlis restituisca una coppia di interi (max_prof,n) dove max_prof è la profondità mas-
sima degli alberi in btlis, mentre n è il numero di alberi in btlis che hanno tale profondità. 

(NOTA: se la lista è vuota, restituisce (0,0))

In [14]:
type btree = 
    | Leaf of int
    | Node of int * btree * btree ;;
    
let bt1 = Node (3, Leaf 5, Node (-4, Leaf 6, Leaf 5)) ;;
let bt2 = Node (1, Node (7, Leaf 6, Leaf 1), Leaf (-2)) 

type btree = Leaf of int | Node of int * btree * btree


val bt1 : btree = Node (3, Leaf 5, Node (-4, Leaf 6, Leaf 5))


val bt2 : btree = Node (1, Node (7, Leaf 6, Leaf 1), Leaf (-2))


In [15]:
let rec depth bt = 
    match bt with 
    | Leaf x -> 0
    | Node(x, left, right) -> 1 + max (depth left) (depth right) ;;

(* Strova massimo in una lista. Si poteva anche farne a meno in realtà... *)
let rec max_elem l = 
    match l with
    | [] -> 0
    | x::l' -> let m = max_elem l' in if (x > m) then x else m ;; 
    
(* Soluzione proposta per max_depth *)
let rec max_depth2 list = 
    match list with
    | [] -> 0
    | bt::list' -> let (d1, d2) = (depth bt, max_depth2 list') in if (d1 > d2) then d1 else d2 ;;

let rec max_depth lis =
    match lis with
    | [] -> 0
    | x::lis' -> max_elem ([depth x]@(List.map depth lis')) ;;
  
let conta_max btlis = 
    match btlis with
    | [] -> (0, 0)
    | btlis' -> let md = max_depth btlis' in 
                let c = List.fold_left (fun acc bt -> if (depth bt = md) then (acc+1) else acc) 0 btlis'
                in (md, c) ;;
      
                    
let btlist = [bt1; bt2] ;;
max_depth2 btlist;; 
conta_max btlist

val depth : btree -> int = <fun>


val max_elem : int list -> int = <fun>


val max_depth2 : btree list -> int = <fun>


val max_depth : btree list -> int = <fun>


val conta_max : btree list -> int * int = <fun>


val btlist : btree list =
  [Node (3, Leaf 5, Node (-4, Leaf 6, Leaf 5));
   Node (1, Node (7, Leaf 6, Leaf 1), Leaf (-2))]


- : int = 2


- : int * int = (2, 2)


**Esercizio 8**

Definire, usando i costrutti di programmazione funzionale in OCaml, una funzione duemax con tipo

duemax : ‘a list -> (‘a * ‘a) option

in modo che duemax lis restituisca Some (x,y) tale che per ogni elemento z di lis vale x ≥ y ≥ z. La
funzione restituisce None in caso di lista vuota. 

Esempi:


duemax [2;3;1;5] = Some (5,3)

duemax [’t’] = Some (’t’,’t’)

duemax [] = None

In [16]:
let rec duemax lis =
    match lis with
    | [] -> None
    | [x] -> Some (x, x)
    | [x; y] -> if (x < y) then Some (x, y) else Some (y, x)
    | x::lis' -> match duemax lis' 
                    with
                    | None -> failwith "Error"
                    | Some (a, b) -> if (a < x) then Some (x, a) else if (x > b) then Some (a, x) else Some (a, b) ;;
                                   
duemax [2;3;1;5] ;;
duemax ['t'] ;;

val duemax : 'a list -> ('a * 'a) option = <fun>


- : (int * int) option = Some (3, 2)


- : (char * char) option = Some ('t', 't')


**Esercizio 9**

Assumendo il seguente tipo di dato che descrive alberi binari di interi:

type btree =
| Void
| Node of int * btree * btree

si definisca, usando i costrutti di programmazione funzionale di OCaml, una funzione distinct con tipo

distinct : btree -> bool

tale che distinct bt restituisca true se i valori interi nell’albero rappresentato da bt sono tutti diversi tra
loro, false se invece bt contiene almeno due numeri uguali.

Ad esempio dato il seguente albero binario:

let bt = Node (3, Node (5, Node(1,Void,Void), Void), Node (-4, Node(6,Void,Void), Node(5,Void,Void)))

abbiamo che distinct bt restituisce false a causa delle due occorrenze del valore 5.

In [17]:
type btree =
| Void
| Node of int * btree * btree ;;

type btree = Void | Node of int * btree * btree


In [18]:
let distinct bt =
    let rec aux t seen =
        match t with
        | Void -> true 
        | Node (x, left, right) -> if List.mem x seen then false else 
                                   let new_node = x::seen in (aux left new_node) && (aux right new_node) 
    in aux bt [] ;;
        
let bt = Node (3, Node (5, Node(1,Void,Void), Void), Node (-4, Node(6,Void,Void), Node(5,Void,Void))) ;;
distinct bt

val distinct : btree -> bool = <fun>


val bt : btree =
  Node (3, Node (5, Node (1, Void, Void), Void),
   Node (-4, Node (6, Void, Void), Node (5, Void, Void)))


- : bool = true


**Esercizio 10**

Si definisca, usando i costrutti di programmazione funzionale di OCaml, una funzione merge con tipo

merge : ’a list list -> ’a list

tale che, data una lista di liste lis, l’applicazione merge lis restituisca una lista contenente tutti gli
elementi delle liste contenute in lis, senza ripetizioni.

Ad esempio, merge [[3;4;5];[9;5];[3;6;9;6]] deve restituire la lista [3;4;5;9;6]. 
L’ordine degli elementi nel risultato può essere qualunque.

In [19]:
let rec merge list = 
    match list with 
    | [] -> []
    | x::lis' -> let result = x @ merge lis' in List.fold_left (fun acc y -> if not (List.mem y acc) then y::acc else acc) [] result;;
    
let l = [[3;4;5];[9;5];[3;6;9;6]] ;;
merge l 

val merge : 'a list list -> 'a list = <fun>


val l : int list list = [[3; 4; 5]; [9; 5]; [3; 6; 9; 6]]


- : int list = [9; 6; 5; 4; 3]


**Esercizio 11**

Definire in OCaml tre funzioni pairs, split e paridispari con tipi

pairs : ’a list -> ’a * ’a list

split : ’a * ’b list -> ’a list * ’b list

paridispari : int list -> int list * int list

La funzione pairs prende una lista [e1;e2;e3;...;ek] di tipo ’a list e restituisce una lista in cui gli
elementi vengono messi in coppia a due a due (il primo con il secondo, il terzo con il quarto, ecc.), come nel
seguente esempio:

pairs [1;2;3;4;5;6] = [(1, 2); (3, 4); (5, 6)]

se la lista passata ha lunghezza dispari, l’ultimo elemento viene ignorato, come nel seguente esempio:

pairs [1;2;3;4;5;6;7] = [(1, 2); (3, 4); (5, 6)]

La funzione split prende una lista di coppie [(a0,b0);(a1,b1);...;(ak,bk)] di tipo ’a * ’b list e
restituisce una coppia di liste contenenti rispettivamente tutti i primi e tutti i secondi elementi delle singole
liste, come nel seguente esempio:

split [(0,5);(1,6);(2,7)] = ([0; 1; 2], [5; 6; 7])

La funzione paridispari prende una lista di interi [n0;n1;n2;...;nk] di tipo int list e restituisce
una coppia di liste (lis1,lis2) di tipo int list * int list in cui lis1 contiene gli elementi in posizioni
pari nella lista di partenza (partendo a contare da 0, ossia [n0;n2;...]) mentre lis2 contiene gli elementi
in posizioni dispari (ossia [n1;n3;...]). 
Ad esempio:

paridispari [4;1;5;7;9;3;2] = ([4;5;9;2],[1;7;3])

In [20]:
let rec pairs lis = 
    match lis with
    | [] -> []
    | [x] -> []
    | [x;y] -> [(x, y)]
    | x::y::lis' -> (x,y)::pairs lis';;
    
let rec split lis = 
    match lis with
    | [] -> ([], [])
    | (x, y)::lis' -> let (first, second) = (split lis') in (x::first, y::second) ;; (* creo due liste e aggiungo elementi *)
    
    
let rec paridispari lis = 
    match lis with
    | [] -> ([], [])
    | [x] -> ([x], [])
    | x::y::lis' -> let (even, odd) = paridispari lis' in (x::even, y::odd) ;; (* come sopra *)
    
pairs [1;2] ;;
pairs [1;2;3;4;5;6;9] ;;
split [(0,5);(1,6);(2,7)] ;;
paridispari [4;1;5;7;9;3;2]

val pairs : 'a list -> ('a * 'a) list = <fun>


val split : ('a * 'b) list -> 'a list * 'b list = <fun>


val paridispari : 'a list -> 'a list * 'a list = <fun>


- : (int * int) list = [(1, 2)]


- : (int * int) list = [(1, 2); (3, 4); (5, 6)]


- : int list * int list = ([0; 1; 2], [5; 6; 7])


- : int list * int list = ([4; 5; 9; 2], [1; 7; 3])


**Esercizio 12**

Assumendo il seguente tipo di dato che descrive alberi n-ari di interi:

type ntree =
| Node of int * ntree list

Definire in OCaml le funzioni contains e remove con i seguenti tipi

contains : int -> ntree -> bool

addchild : int -> ntree -> ntree


tali che:
  - contains prende un intero n e un albero t, e restituisce true se n è presente in t. Restituisce false altrimenti
  
  
  - addchild prende due interi n e m e un albero t, e restituisce un albero ottenuto aggiungendo ad ogni  nodo etichettato con n in t, un figlio foglia etichettato con m. Se nessun nodo è etichettato con n, il metodo addchild restituisce un albero identico a t.

In [21]:
type ntree = 
    | Node of int * ntree list ;;

type ntree = Node of int * ntree list


In [22]:
let t = Node (1, [Node (2,[]);Node (3,[Node (4,[]);Node (5,[])]);Node (6,[Node (3,[])])]) ;;

val t : ntree =
  Node (1,
   [Node (2, []); Node (3, [Node (4, []); Node (5, [])]);
    Node (6, [Node (3, [])])])


In [23]:
let rec contains n t = 
    match t with 
    | Node (x, []) -> if (n = x) then true else false 
    | Node(x, children) -> if (n=x) then true else List.exists (contains n) children ;;
    
(* Soluzione proposta *)
let rec addchild n m t = 
    match t with
    | Node (x, children) -> if (x = n) then Node (x, (Node (m, [])))::List.map (addchild n m) children 
                            else Node(x, List.map addchild n m children) ;; 


contains 6 t ;;
contains 8 t ;;
addchild 1 4 (Node(1, []))

val contains : int -> ntree -> bool = <fun>


error: compile_error

**Esercizio 13**

Definire in OCaml tre funzioni coppie_da_lista, lista_tripla e espandi con tipi

coppie_da_lista : ’a list -> ’a * ’a list

lista_tripla : ’a * ’b * ’c list -> ’a list * ’b list * ’c list

espandi : ’a * int list -> ’a list list

La funzione coppie_da_lista prende una lista [e1;e2;e3;...;ek] di tipo ’a list e restituisce una
lista contenente tutte le possibili coppie che si possono formare prendendo due elementi qualunque della
lista, come nel seguente esempio:

coppie_da_lista [1;2;3;4] = [(1, 2); (1, 3); (1, 4); (2, 3); (2, 4); (3, 4)]

La funzione lista_tripla prende una lista di triple [(a1,b1,c1);(a2,b2,c2);...;(ak,bk,ck)] di
tipo ’a * ’b * ’c list e restituisce una tripla di liste contenenti rispettivamente tutti i primi, tutti i
secondi e tutti i terzi elementi delle singole liste, come nel seguente esempio:

lista_tripla [(1,5,4);(2,6,3);(3,7,2)] = ([1; 2; 3], [5; 6; 7], [4; 3; 2])

La funzione espandi prende una lista di coppie [(e1,n1),(e2,n2),...,(ek,nk)] di tipo (’a * int) list
e restituisce una lista di tipo ’a list list in cui ogni coppia (ei,ni) è sostituita da un lista con ni copie
dell’elemento ei. 

Ad esempio:
espandi [(’a’,4);(’b’,3);(’c’,5)] =
[[’a’; ’a’; ’a’; ’a’]; [’b’; ’b’; ’b’]; [’c’; ’c’; ’c’; ’c’; ’c’]]

In [24]:
let rec coppie_da_lista lis =
    match lis with
    | [] -> []
    | x::lis' -> (List.map (fun y -> (x,y)) lis') @ (coppie_da_lista lis') ;; 

let rec lista_tripla lis = 
    match lis with
    | [] -> ([], [], [])
    | (x,y,z)::lis' -> let (first,second,third) = lista_tripla lis' in (x::first, y::second, z::third) ;;
    
    
let espandi lis = List.fold_left (fun acc (e, c) -> if (c>0) then acc@[List.init c (fun _ -> e)] else []) [] lis ;;

coppie_da_lista [1;2;3;4] ;;
lista_tripla [(1,5,4);(2,6,3);(3,7,2)] ;;
espandi [('a',4);('b',3);('c',5)]

val coppie_da_lista : 'a list -> ('a * 'a) list = <fun>


val lista_tripla : ('a * 'b * 'c) list -> 'a list * 'b list * 'c list = <fun>


val espandi : ('a * int) list -> 'a list list = <fun>


- : (int * int) list = [(1, 2); (1, 3); (1, 4); (2, 3); (2, 4); (3, 4)]


- : int list * int list * int list = ([1; 2; 3], [5; 6; 7], [4; 3; 2])


- : char list list =
[['a'; 'a'; 'a'; 'a']; ['b'; 'b'; 'b']; ['c'; 'c'; 'c'; 'c'; 'c']]


**Esercizio 14**

Considerando il tipo ’a colored_tree che rappresenta alberi con nodi colorati definito
nell’esercizio 1, scrivere una funzione raccogli che, dato un albero di questo tipo, restituisce una tripla di
liste (l1,l2,l3) di tipo ’a list * ’a list * ’a list dove l1 contiene tutti i valori contenuti nei nodi
Black, l2 tutti i valori nei nodi Red ed l3 tutti i valori nei nodi Blue. 

Ad esempio, dato il seguente albero:

let tree = Red ("a",
Blue ("b",
Black "c",
Red ( "d", Black "e", Black "f")),
Red ( "g",
Blue ( "h", Black "i", Black "j"),
Black "k")) ;;


l’applicazione raccogli tree deve dare come risultato:
(["c"; "e"; "f"; "i"; "j"; "k"], ["a"; "d"; "g"], ["b"; "h"])
L’ordine degli elementi nelle liste può essere diverso rispetto a questo esempio.

In [25]:
type 'a colored_tree = 
    | Black of 'a
    | Red of 'a * ('a colored_tree) * ('a colored_tree) 
    | Blue of 'a * ('a colored_tree) * ('a colored_tree) ;;

type 'a colored_tree =
    Black of 'a
  | Red of 'a * 'a colored_tree * 'a colored_tree
  | Blue of 'a * 'a colored_tree * 'a colored_tree


In [26]:
let rec raccogli t = 
    match t with 
    | Black x -> ([x], [], []) 
    | Red (x, left, right) -> let (black,red,blue) = raccogli left in
                              let (bl, r, blu) = raccogli right in
                              (bl@black, x::r@red, blu@blue) 
    | Blue (x, left, right) -> let (black,red,blue) = raccogli left in
                               let (bl, r, blu) = raccogli right in
                               (black@bl, red@r, x::blue@blu) ;;

let tree = Red ("a", Blue ("b", Black "c", Red ( "d", Black "e", Black "f")), Red ( "g", Blue ( "h", Black "i", Black "j"), Black "k")) ;;
raccogli tree 

val raccogli : 'a colored_tree -> 'a list * 'a list * 'a list = <fun>


val tree : string colored_tree =
  Red ("a", Blue ("b", Black "c", Red ("d", Black "e", Black "f")),
   Red ("g", Blue ("h", Black "i", Black "j"), Black "k"))


- : string list * string list * string list =
(["k"; "i"; "j"; "c"; "f"; "e"], ["a"; "g"; "d"], ["h"; "b"])


**Esercizio 15**

Esercizio 2.2.
Assumendo il seguente tipo di dato che descrive alberi binari di interi:

type btree =
| Void
| Node of int * btree * btree

si definisca, usando i costrutti di programmazione funzionale di OCaml, una funzione flat con tipo

flat : btree -> int list list

tale che flat bt restituisca una lista contenente le liste di valori presenti ad ogni livello di profondità
dell’albero.

Ad esempio dato il seguente albero binario (a destra in una rappresentazione visuale):

let bt =Node (3,Node (5,Node(1,Void,Void),Void),Node (-4,Node(6,Void,Void),Node(8,Void,Void)))

abbiamo che flat bt restituisce [[3],[5,-4],[1,6,8]].

In [27]:
type btree =
    | Void
    | Node of int * btree * btree

type btree = Void | Node of int * btree * btree


In [62]:
(* Soluzione proposta *)
let rec flat bt =
    let rec merge lis1 lis2 =
        match lis1,lis2 with
        | [],_ -> lis2
        | _,[] -> lis1
        | x::lis1',y::lis2' -> (x@y)::(merge lis1' lis2')
        in
        match bt with
        | Void -> []
        | Node (n,bt1,bt2) -> let lis1 = flat bt1 in let lis2 = flat bt2 in [[n]]@(merge lis1 lis2) ;;

                                
let bt = Node (3,Node (5,Node(1,Void,Void),Void),Node (-4,Node(6,Void,Void),Node(8,Void,Void))) ;;
flat bt ;;

val flat : btree -> int list list = <fun>


val bt : btree =
  Node (3, Node (5, Node (1, Void, Void), Void),
   Node (-4, Node (6, Void, Void), Node (8, Void, Void)))


- : int list list = [[3]; [5; -4]; [1; 6; 8]]
